<hr style="border:0.2px solid black"> </hr>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems

<figure>
  <IMG SRC="img/ntnu_logo.png" WIDTH=250 ALIGN="right">
</figure>

# <ins>Example:</ins> Regression analysis
*Developed by David Steffelbauer*

*(based on two notebooks that were developed with Mark Bakker for his TU Delft course on Exploratory Computing with Python)*
    
Version 1.0
    
<hr style="border:0.2px solid black"> </hr>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In this Notebook, we learn how to fit a model to a set of data. In the first part of this Notebook, we fit linear models to a dataset. Then, we look under the hood of these regression analyses, we discuss how the best parameters are computed, how the goodness of fit can be quantified, and what these other parameters are that some of the regression functions return. We discuss the problem of overfitting by fitting polynomials of different degree on the data. Finally, we learn how to fit arbitrary functions on data.

### Root mean square error
One way to quantify the fit between data and a model is to compute the root mean square error. The error is defined as the difference between the observed value and the modeled value. Another term for the error is the residual. If the error of data point $i$ is written as 
$$\varepsilon_i = y_i - \hat{y}_i$$ 
where $y_i$ is the measured and $\hat{y}_i$ is the fitted value  and the total number of observations is $N$, then the sum of squared errors $S$ is

$$S = \sum{\varepsilon_i^2}$$ 

When the total number of observations is $N$, the root mean square error $E$ is computed as

$$E=\sqrt{\frac{1}{N}S}=\sqrt{\frac{1}{N}\sum{\varepsilon_i^2}}$$

The root mean square error is an estimate of the goodness of fit and can be computed for any model and any dataset.

### Exercise 1. Fit a straight line
Load the $x,y$ values of 30 data points from the file `xydatafit.dat` with `np.loadtxt` function. Fit a straight line through the data using the `linregress` function of `scipy.stats`. Note that the `linregress` function returns 3 other values beyond the slope and intercept (use `linregress?` to find out); more on these 3 additional values later on in this Notebook. Plot the data (with markers only) and add the fitted straight line. Add a legend. Generate a Python function called `rmse` that computes the root mean squared error between the fitted values and the real data (this function will be used later on a lot of times). Add the root mean square error as a title to the graph. Print the optimal values for the slope and intercept of the straight line to the screen.

In [ ]:
from scipy.stats import linregress

def rmse(y1, y2):
    return np.sqrt(np.sum((y1 - y2) ** 2) / len(y1))
    
xdata, ydata = np.loadtxt('data/xydatafit.dat')
slope, intercept, r_value, p_value, std_err = linregress(xdata, ydata)
yfit = slope * xdata + intercept
plt.figure()
plt.plot(xdata, ydata, 'bo', label='observed')
plt.plot(xdata, yfit, 'r', label='fit')
E = rmse(yfit, ydata)
plt.title('RMSE: '+str(E))
plt.legend(loc='best')
plt.xlabel('x', fontsize=14)
plt.ylabel('y', fontsize=14)
print('slope, intercept:', slope, intercept)

## Least squares
In the exercises above, the *optimal* or *best* parameters were obtained with the `linregress` method. But how does these method do that? Or maybe a more fundamental question: 'What is *optimal*?' or 'What is *best*?' In this Notebook, we define *best* as the parameter set that minimizes the sum of the squared errors $S$ (so it also minimizes the root mean square error $E$). Such an optimization approach is referred to as a *least squares* approach. 

We can try to fit a straight line through the data points, but you can already see that the  points don't lie on a line, so there is no straight line that goes exactly through all the points. The straight line is written as $y=ax+b$, where $a$ is the slope of the line and $b$ is called the intercept (it is the value of $y$ for $x=0$). We write a function that takes as input arguments an array of observed $x$ values and an array of corresponding $y$ values, and values for the slope $a$ and intercept $b$. The function returns the sum of squared errors, where the error is defined as the difference betweeen the observed value of $y$ and the value of the straight line at that same $x$ value. The equation for the error at point $i$ is $\varepsilon_i$ and may be written as

$$\varepsilon_i = y_i - (ax_i + b)$$

In [ ]:
def sse(a, b, x=xdata, y=ydata):
    error = y - (a * x + b)
    return np.sum(error ** 2)

As you can see, different values of $a$ and $b$ give different values for the sum of squared errors `sse`. The `sse` for $a=-0.05$ and $b=0.3$ is larger than for $a=-0.02$ and $b=0.5$. 

In [ ]:
print('sse of a=-0.05, b=0.3:', sse(a=-0.05, b=0.3))
print('sse of a=-0.02, b=0.5:', sse(a=-0.02, b=0.5))

What we can do is compute the `sse` function for a larger number of $a$ and $b$ values. If we do that on a regular grid, we can create contours of the `sse` function. The `sse` function is constant along any contour. A contour map of the `sse` function is similar to an elevation map. The goal is now to find the combination of $a$ and $b$ that gives the smallest value of the sum of squared errors. In the graph below, you can see that the smallest value of `sse` is obtained at $a\approx -0.03$, $b\approx 0.4$.

In [ ]:
a, b = np.meshgrid(np.linspace(-0.05, -0.02, 50), np.linspace(0.3, 0.5, 50))
ssevec = np.vectorize(sse)
z = ssevec(a, b)
plt.figure()
plt.contour(a, b, z, 100)
plt.colorbar()
plt.xlabel('a')
plt.ylabel('b');

How do we minimize the sum of squared errors? As usual, we find the minimum of a function by taking the derivative and setting it to zero. This is a little involved, but not too difficult. The sum of squared errors is written as $S$

$$
S=\sum_{i=1}^N\varepsilon_i^2=
\sum_{i=1}^N[y_i-(ax_i+b)]^2
$$

where $N$ is the number of observations. The slope $a$ and intercept $b$ are determined such that $S$ is minimized, which means that the following derivatives are zero

$$\frac{\partial S}{\partial a}=0 \qquad \frac{\partial S}{\partial b}=0$$

Differentiation gives (using the chain rule)

$$
\frac{\partial S}{\partial a}=\sum_{i=1}^N[2(y_i-ax_i-b)(-x_i)]=
2a\sum_{i=1}^Nx_i^2+2b\sum_{i=1}^Nx_i-2\sum_{i=1}^Nx_iy_i
$$

$$
\frac{\partial S}{\partial b}=\sum_{i=1}^N[2(y_i-ax_i-b)(-1)]=
2a\sum_{i=1}^Nx_i+2bN-2\sum_{i=1}^Ny_i
$$

Setting the derivatives equal to zero and division by 2 gives

$$
a\sum_{i=1}^Nx_i^2+b\sum_{i=1}^Nx_i-\sum_{i=1}^Nx_iy_i=0
$$

$$
a\sum_{i=1}^Nx_i+bN-\sum_{i=1}^Ny_i=0
$$

This system of two linear equations with two unknowns ($a$ and $b$) may be solved to give

$$ a=\frac{N\sum_{i=1}^Nx_iy_i-\sum_{i=1}^Nx_i\sum_{i=1}^Ny_i}
{N\sum_{i=1}^Nx_i^2-\sum_{i=1}^Nx_i\sum_{i=1}^Nx_i}
$$

$$
b=\bar{y}-a\bar{x}
$$
where $\bar{x}$ and $\bar{y}$ are the mean values of $x$ and $y$, respectively. 

### Exercise 2. Fitting a straight line revisited
Compute the optimal values (in the least squares sense) of $a$ and $b$ using the two equations derived above and the corresponding sum of squared errors (using the `xdata` and `ydata` arrays for the data points given above). Next, use the `linregress` function of the `scipy.stats` package to compute the optimal values and verify that the `linregress` function gives the same answers.

In [ ]:
N = len(xdata)
a = (N * np.sum(xdata * ydata) - np.sum(xdata) * np.sum(ydata) ) / \
    (N * np.sum(xdata ** 2) - np.sum(xdata) * np.sum(xdata))
b = np.mean(ydata) - a * np.mean(xdata)
print('optimal values of a and b:', a, b)
print('sse:', sse(a, b))

In [ ]:
from scipy.stats import linregress
slope, intercept, r, p, s = linregress(xdata, ydata)
print('slope and intercept according to linregress:', slope, intercept)

### The correlation coefficient, $p$-value and standard error. 
The `linregress` function returns 5 values. Besides the slope and intercept, these are somewhat cryptically defined as the correlation coefficient, the $p$-value, and the standard error. Each of these three values are a quantification of the goodness of fit. According to statisticians, these terms in the `scipy.stats` documentation are somewhat imprecisely defined (they will likely be updated in the future). Let's discuss the correlation coefficient.

The square of the correlation coefficient $r$ is the *r-squared value* and is defined as

$$r^2 = 1 - \sum{(y_i - \hat{y}_i)^2} \left/ \sum{(y_i - \bar{y})^2} \right. $$

where $y_i$ is the $y$ value of data point $i$, while $\hat{y}_i$ is the fitted values at data point $i$. It can also be written as 

$$r^2 = \frac{\text{var}(y) - \text{var}(y-\hat{y})}{\text{var}(y)}$$

So the $r^2$ value is the variance of $y$ minus the variance of the remaining residuals (the data values minus the fitted values), divided by the variance of $y$, and is also referred to as the 'percentage of variance explained'. If the model goes exactly through the data (a perfect fit), then the variance of the residuals is zero, and $r^2=1$. If the model doesn't do much better than simply the mean of $y$, then the  $r^2$ is very close zero. A value of $r^2$ close to 1 is generally a good thing, but it is not possible to say anything definitive about the goodness of fit by just reporting the $r^2$ value (although many people do).

### Exercise 3. Verification of correlation coefficient
Implement the equations for $r^2$ given above to verify that the values returned by the `linregress` function are correct. 

In [ ]:
from scipy.stats import linregress

slope, intercept, r, p, s = linregress(xdata, ydata)
yfit = slope * xdata + intercept

print('r squared according to formula:', end=' ')
print(1 - sum((ydata - yfit) ** 2) / sum((ydata - np.mean(ydata)) ** 2))
print('r squared according to linregress:', r**2)

### Meaning of the $p$-value
The fourth return value of the `linregress` function is the $p$-value. The $p$-value is related to the question whether the estimated slope is significantly different from zero. When the slope is significantly different from zero, you can state that there is a linear relationship between the two variables. The $p$-value is related to the question whether the estimated absolute value of the slope is significantly different from zero when you perform a $t$-test. When the $p$-value is less than 0.05, this means that when you perform a two-sided $t$-test you can reject the null hypothesis that the slope is zero in favor of the alternative hypothesis that the slope is not zero. In layman terms: it means that there is less than 5% chance that the slope is zero and more than 95% chance that the slope is not zero. Or even simpler: the slope is significantly different from zero. The $p$-value is computed from a $t$ distribution with mean zero, standard deviation equal to the standard deviation of the fitted slope, and number of degrees of freedom equal to $N-2$. The $p$-value is the two-sided probability that the slope is larger than the fitted slope. 

In [ ]:
slope, intercept, r, p, s = linregress(xdata, ydata)
print('computed p-value and standard error:', p, s)

As the fitted slope is above zero, the probability that the true slope is larger than the fitted slope can be computed as (note that the second argument equals $N-2=30-2=28$ for this case): 

In [ ]:
from scipy.stats import t
v = len(ydata) - 2
1 - t.cdf(abs(slope), v, loc=0, scale=s)

The $p$-value is the two-sided probability, which can now be computed as

In [ ]:
print('p-value from t-distribution:', 2 * (1 - t.cdf(abs(slope), v, loc=0, scale=s)))

Obviously, there is a high probability that the slope is significantly different from zero.

## Real data - Minimum Night Flow analysis
The csv file `inflow_data.csv` contains real data that consists of inflow measurements in a water distribution system from a small town in Austria last February. Inflow in a system is an important measure since it represents the total water consumption of all households in a certain area. It also contains data of water losses as well.

The first column  contains the timestamps when the measurements are taken, the second column contains the inflow measurements in liters per second (L/s). 

We can load this data with Panda's `read_csv` function. Additionally, we specify that `time` should be used as index. Since the index column contains timestamps, we force `pandas` to parse this data as `datetime` objects. This will enable Pythons `datetime` functionalities that we can use later on.

In [ ]:
import pandas as pd

Q = pd.read_csv('data/inflow_data.csv', index_col='time', parse_dates=True)

Now we can plot the whole dataset with Panda's plot function in following way:

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
Q.plot(ax=ax);
plt.ylabel('Inflow  (L/s)');

We see that the water consumption follows a certain pattern with repeating elements with a frequency of 24 hours. On the third day, we can see a large spike in the consumption due to high water usage. During the nighttime on the 6th of Febrary, the flow seems to be larger compared to the other nights. This can be also seen on the 24th and the 26th. 

Let's have a closer look on the first day of the dataset to reveal the repeating pattern. This can be done by using a datetime-like object, in this case a string, defining the specific date that we want to look on. Note that this works only for a datetime index.

In [ ]:
Qday = Q['2017-2-1']
Qday.plot();

The water consumption follows strictly the daily routine of the inhabitants in a zone. At around 6:00 in the morning, people wake up, have a shower, go on the toilet or prepare breakfast–resulting in the so-called morning peak. After that, some people will go to work and leave the measurement zone. Hence, the water consumption decreases. In the evening, people return to their homes. Again, more water is used again, resulting in a second peak. For example,  people prepare dinner, take showers, do the laundry or dishes, use the toilet and brush their teeth before they finally go to sleep. While people are sleeping, they do not use water. This can be seen between 2:00 and 4:00 in the night, which is called the minimum night flow (`MNF`) hours.

The `MNF`-hours are very important for water utilities. Since the consumption is minimal, the pressure in the system is high. Leakage outflow is pressure dependent: The higher the pressure, the more water is lost through a leak. That is why leaks can be detected best during these hours. Additionally, fluctuations in the inflow measurements, which result from the stochastic water usage behaviour of the inhabitants, are minimal during the `MNF`.

Let's have a closer look on the water consumption during the `MNF`. With the `between_time` function, we can choose measurements during the `MNF`. The daily mean `MNF` can be computed by grouping the data according to the day of the month.

In [ ]:
Qnight = Q.between_time('02:00', '04:00')
MNF = Qnight.groupby(Qnight.index.day).mean()
MNF.columns = ['MNF']

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111)
MNF.plot(kind='bar', ax=ax)
M = MNF.mean().values
E = MNF.std().values
plt.axhline(M, color='k', label=r'$\mu$')
plt.axhline(M+E, color='k', linestyle='--', label=r'$\mu \pm \sigma$')
plt.axhline(M-E, color='k', linestyle='--')
plt.axhline(M+2*E, color='k', linestyle=':', label=r'$\mu \pm 2\sigma$')
plt.axhline(M-2*E, color='k', linestyle=':')

plt.xlabel('day');
plt.ylabel('MNF  (L/s)');
plt.legend(loc='upper left');

We can see again the days of higher consumption (February the 6th, 24th and 26th). 

Water utilities analyse the `MNF` to find leaks in their systems. If the consumption during `MNF`  is higher than a certain value (e.g. higher than the mean plus two standard deviations) for a couple of consecutive days, a leak might appeared in the system. The utility will take counter measures through sending a team to the measurement zone to find and repair this leak.

Not all leaks result in an sudden increase in water consumption. Some leaks grow slowly over time. These leaks are harder to find with statistical methods, because of their insidious increase in flow. They will increase the mean and standard deviation of the `MNF` measurements. That is where the linear regression analysis joins the game.

### Exercise 4. Determine linear trends of real data - a simple $p$-value based leak detector
The `MNF` of the example above is saved the file `night_minimum.dat`. Load this file with `np.loadtxt`. Split the data in two halves. Perform a linear regression on each half and print the slope as well as the p-value to the screen. Determine if there is a growing leak in the system in the first as well as in the second half of the data through a $p$-value test.

In [ ]:
from scipy.stats import linregress

def leak_detector(x, y):
    slope, intercept, r_value, p_value, sig_a = linregress(x, y)
    output = '\tslope=' + str(slope) + ', p-value=' + str(p_value)
    if p_value < 0.05:
        output += ' => There is a LEAK !!!'
    else:
        output += ' => No Leak, everything is fine ...'
    return output
    
xdata, ydata = np.loadtxt('data/night_minimum.dat')

half = round(len(xdata)/2)
output1 = leak_detector(xdata[:half], ydata[:half])
output2 = leak_detector(xdata[half:], ydata[half:])

print('First half:', output1)
print('Second half:', output2)

## How accurate are our parameter estimates?

Let's get back to the first example dataset in file `xydatafit.dat`.  Till now, we have seen how the accuracy of a model can be quantified by goodness of fit measurements like the (i) root mean squared error or (ii) the sum of squared errors. Additionally, we have seen how optimal parameters can be found through least squares optimisation. A linear model was fitted by searching for the parameters (slope and intercept) that gave the best fit, where the best fit was defined as the smallest possible value of the sum of the squares of the errors (the residuals). 

We have to be aware that the optimal parameters are estimates of the true underlying parameters. Since the optimal parameters are estimates, not only their optimal values should be reported, but also their (estimated) confidence intervals. 

The confidence intervals of the parameters of a model can be estimated if we can calculate the standard deviation of the estimates of the parameters. Additionally, a few other statistical conditions have to be met, especially that the errors are normally distributed and independent (the error at one point does not depend on the error at another point), and that 
the number of observations $N$ is sufficiently large. A common value for large is 40 or more. 

### Confidence intervals of parameters of a straight line

The standard error returned by the `linregress` model is the estimated standard deviation of the fitted slope $s_a$. The standard deviation of the slope should be interpreted similar to the standard deviation of the mean. The computed slope is a statistical value so it has an estimated standard deviation.  The equation is
$$s_a = \sqrt{\frac{\sum(y_i-\hat{y}_i)^2}{N-2}} \left/ \sqrt{\sum{(x_i-\bar{x})^2}} \right.$$
where $\hat{y}_i$ are the fitted values.

The standard error of the intercept $s_b$ can be computed from $s_a$ in the following way

$$s_b = s_a \sqrt{\frac{1}{N}\sum{x_i^2}}$$

The 95% confidence interval may now be estimated as plus or minus twice the estimated standard deviation. 

### Exercise 5. Confidence intervals of the parameters of a straight line using estimated standard deviations
Load the $x,y$ values of 30 data points from the file `xydatafit.dat`. Fit a straight line through the data by minimizing the sum of squared errors (using the `linregress` function of `scipy.stats`). Estimate the 95% confidence intervals of the slope and intercept of the straight line using the estimates of the standard deviations of the parameters as given above. 

In [ ]:
from scipy.stats import linregress
xdata, ydata = np.loadtxt('data/xydatafit.dat')
a, b, r_value, p_value, s_a = linregress(xdata, ydata)
print('estimated slope:', a)
print('estimated 95% confidence interval of the slope:', a - 2 * s_a, a + 2 * s_a)
s_b = s_a * np.sqrt(sum(xdata ** 2) / len(xdata))
print('estimated intercept:', b)
print('estimated 95% confidence interval of the intercept:', b - 2 * s_b, b + 2 * s_b)

### The bootstrap method

What now if we don't have errors that are normally distributed? Or if we have only a few data points (like above, where we only have 30)? Or if we don't have equations to compute the standard deviations of the parameters, for example when we want to fit something else than a straight line? One option is to use a method called bootstrapping. 

Bootstrapping is a powerful statistical technique to estimate confidence intervals. It is especially useful when the sample size that we are working with is small. Bootstrap techniques work quite well with samples that have less than 40 elements while assuming nothing about the distribution of the data (this information is taken from [here](http://statistics.about.com/od/Applications/a/Example-Of-Bootstrapping.htm)).

A great advantage of bootstrap is its simplicity. It is a straightforward way to derive confidence intervals for estimators of complicated parameters of the distribution. Although for most problems it is impossible to know the true confidence intervals, bootstrap is asymptotically more accurate than the standard intervals obtained using sample variance and assumptions of normality.
Although bootstrapping is (under some conditions) asymptotically consistent, it does not provide general finite-sample guarantees. There are serveral underlying assumptions for bootstrapping to be valid, including that the errors are independent (this information is taken from <a href="https://en.wikipedia.org/wiki/Bootstrapping_(statistics)">here</a>).

There are many different flavors of bootstrapping. In this Notebook, we will make use of *resampling residuals* to estimate the confidence intervals of the estimated parameters. The procedure works as follows.

Consider a data set with $N$ values $x_i$ and corresponding values $y_i$. A model is fitted to the data and the fitted values are called $\hat{y}_i$. The residual $r_i$ can be computed as $r_i=y_i - \hat{y}_i$. The confidence intervals of the parameters of the model may be obtained with bootstrapping as follows:
1. Compute and store the optimal parameters using the regular approach. 
2. Compute the fitted values $\hat{y}_i$.
3. Compute and store the residuals $r_i$ in an array.
4. For every data point, draw a residual at random (with replacement) from the array of residuals using the `numpy.random.choice` function and add it to the fitted value $\hat{y}_i$. Let's call these values $\tilde{y}_i$.
5. Compute and store the optimal parameters that fit through the $\tilde{y}_i$ values.
6. Repeat steps 4 and 5 $M$ times, for example using $M=1000$. 
7. Compute the approximate $95$% confidence intervals of the parameters using the `np.percentile` function.

### Exercise 6. Confidence intervals of the parameters of a straight line using bootstrapping
Load the $x,y$ values of 30 data points from the file `xydatafit.dat`. Fit a straight line through the data by minimizing the sum of squared errors (using the `linregress` function of `scipy.stats`). Estimate the 95% confidence intervals of the slope and intercept of the straight line using bootstrapping and $M=1000$. Create a histogram of the 1000 estimates of the slope and a separate histogram of the 1000 estimates of the intercept. Report the 95% intervals of both parameters. How do they compare to the estimates in Exercise 5?

On the same histograms, plot the theoretical Normal distributions using your estimates of the slope, intercept, and their standard deviations from Exercise 1. Spoiler: they should match pretty well.

In [ ]:
from scipy.stats import linregress
xdata, ydata = np.loadtxt('data/xydatafit.dat')
slope, intercept, r_value, p_value, sig_a = linregress(xdata, ydata)
yfit = slope * xdata + intercept
res = ydata - yfit
#
N = len(xdata)
M = 1000
err = np.random.choice(res, size=(M, N), replace=True)
a = np.zeros(M)
b = np.zeros(M)
for i in range(M):
    a[i], b[i], r, p, std = linregress(xdata, yfit + err[i])
    
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(121)
plt.hist(a, bins=20, density=True)
plt.title('Histogram of slope')
ax2 = fig.add_subplot(122)
plt.hist(b, bins=20, density=True)
plt.title('Histogram of intercept')
print('95% interval of slope:', np.percentile(a, [2.5, 97.5]))
print('95% interval of intercept:', np.percentile(b, [2.5, 97.5]))

# last question
from scipy.stats import norm
sig_b = sig_a * np.sqrt(sum(xdata ** 2) / len(xdata))
xs = np.linspace(-0.05,-0.01,100)
ys = norm.pdf(xs, loc=slope, scale=sig_a)

ax1.plot(xs, ys, 'r')
xs = np.linspace(0.3,0.5,100)
ys = norm.pdf(xs, loc=intercept, scale=sig_b)
ax2.plot(xs, ys, 'r');

Of course, linear functions are not the only possible functions that can be fitted on data. In this and the next examples we will focus on fitting more complicated functions. First, we will start by fitting polynomials.

### Exercise 7. Fit a polynomial
Use the $x,y$ values of 30 data points from the file `xydatafit.dat`. Fit a second degree polynomial (a parabola–degree $D=2$) through the data using the `np.polyfit` function. Plot the data and the fitted parabola. Add a legend. Report the root mean squared error in the title. Compare the root mean squared error to the linear fit. Did the root mean squared error improve?

In [ ]:
xdata, ydata = np.loadtxt('data/xydatafit.dat')
a, b, c = np.polyfit(xdata, ydata, 2)
yfit = a * xdata ** 2 + b * xdata + c
plt.figure()
plt.plot(xdata, ydata, 'bo', label='observed')
plt.plot(xdata, yfit, 'r', label='fit')
E = rmse(yfit, ydata)
plt.legend(loc='best')
plt.title('RMSE: '+str(E));

## The problem of overfitting
In the former example, we have seen that a more complex model, e.g. a polynomial of degree $D=2$, represents the data better than a linear model. The reason for that is that for a linear model, the least squares algorithm can only tune two parameters (slope and intercept) to fit the data, whereas it has the freedom to tune an additional parameter–in total three parameters–for a polynomial of degree $D=2$. So what happens, if we increase the complexity of our models by adding more fitting parameters? In general a polynom of degree $D$ has $D+1$ fitting parameters. Do our models get better with increasing complexity?

### Exercise 8. Increase the polynomial degree
Use the $x,y$ values of 30 data points from the file `xydatafit.dat`. Use a for loop to fit polynomials of increasing degree starting from $D=1$ (straight line equal to the linear regression fit from before) to $D=8$ through the data. Use `numpy`'s `polyval` function to compute the fitted `yfit` parameters, so you don't have to type in all polynomial equations yourself. Print the polynomial degree $D$ and its corresponding root mean square error for each fit. Plot the data for the polynomial with the highest fitted degree ($D=8$). What happens to the root mean squared error in dependency on the polynomial degree $D$? Does the polynomial with $D=8$ describe the data better than the one with $D=2$?

In [ ]:
for i in range(1, 9):
    p = np.polyfit(xdata, ydata, i)
    yfit = np.polyval(p, xdata)
    E = np.sqrt(np.sum((yfit - ydata) ** 2) / len(ydata))
    
    print(f'D={i} => RMSE={E:.5f}')

plt.figure()
plt.plot(xdata, ydata, 'bo', label='observed')
plt.plot(xdata, yfit, 'r', label='polyfit deg. ' + str(i))
plt.legend(loc='best');

Obviously, higher polynomial degrees (in other words more complex models) result in better fits with smaller (root mean squared) errors. The first dataset was obtained through one experiment. Now, the experiment is repeated with completely the same settings to obtain a new dataset `xydatafit_repeated.dat`. Does the high order polynomial we have found on the former dataset describe the new dataset as good as before?

### Exercise 9. Old fits on new data
Fit two polynomial of degree $D=2$ and $D=8$ on the old dataset (`xydatafit.dat`). Then use the fitted parameters obtained from the old dataset on new measurement data (`xydatafit_repeated.dat`) (without refitting the polynomials!) to compute the root mean square error. Print the root mean square error for each polynomial to the screen. Which polynomial performs best? Plot the new data as circles, the old data as xs and both polynomials as lines in one figure.

In [ ]:
xdata, ydata = np.loadtxt('data/xydatafit.dat')


p2 = np.polyfit(xdata, ydata, 2)
y2 = np.polyval(p2, xdata)


p8 = np.polyfit(xdata, ydata, 8)
y8 = np.polyval(p8, xdata)

xdata_r, ydata_r = np.loadtxt('data/xydatafit_repeated.dat')

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(xdata_r, ydata_r, 'o', label='new_data')
ax.plot(xdata, ydata, 'kx', label='old_data')

ax.plot(xdata_r, y2, 'r-', label='fit D=2')
ax.plot(xdata_r, y8, 'b-', label='fit D=8')
plt.legend()

rmse2 = rmse(y2, ydata_r) #np.sqrt(np.sum((y2 - ydata_r) ** 2) / len(ydata_r))
rmse8 = rmse(y8, ydata_r) #np.sqrt(np.sum((y8 - ydata_r) ** 2) / len(ydata_r))

print('Root mean square errors:')
print('D=2, RMSE=', rmse2)
print('D=8, RMSE=', rmse8)


What happened? The polynomial of high order–fitting the data of the first experiment so well–completely failed on the new dataset, although the experimental settings were the same. Now, a simple polynomial of low order ($D=2$) fits the new data a lot better than the high order polynomial ($D=8$), which can be seen in its much smaller root mean squared error.  

This problem is called overefitting. In regression analysis, overfitting occurs frequently.
Overfitting is the use of models or procedures that violate <a href="https://en.wikipedia.org/wiki/Occam%27s_razor">Occam's razor</a>, for example by including more adjustable parameters than are ultimately optimal. There are too many adjustable parameters, for example, consider a dataset where fit data for y can be adequately predicted by a linear function. Such a function requires only two parameters (the intercept and the slope). Replacing this simple function with a new, more complex quadratic function, carries a risk: Occam's razor implies that any given complex function is a priori less probable than any given simple function. If the new, more complicated function is selected instead of the simple function, and if there was not a large enough gain in data fit to offset the complexity increase, then the new complex function "overfits" the data, and the complex overfitted function will likely perform worse than the simpler function on other datasets, even though the complex function performed as well, or perhaps even better, on the original dataset on which it was fitted.
As an extreme example, if there are $N$ parameters in a linear regression with $N$ data points, the fitted line can go exactly through every point. 

In contrast to overfitting, underfitting occurs when a model cannot adequately capture the underlying structure of the data. So the goal is to find a model that describes in an ideal way with the right amount of complexity.
(Explanations taken from <a href="https://en.wikipedia.org/wiki/Overfitting">here</a>)

### Other goodness of fit measures

There exist goodness of fit measurements (apart from root mean squared error and similar measures) that are capable to take the complexity of the model into account. We will have a closer look on two measures, the 
<a href="https://en.wikipedia.org/wiki/Akaike_information_criterion">Akaike information criterion</a> (`AIC`) and <a href="https://en.wikipedia.org/wiki/Bayesian_information_criterion">Bayesian information criterion</a> (`BIC`)

$$AIC= N \cdot \ln(\hat{\mathcal{L}}) + 2 \cdot k$$

$$BIC= N \cdot \ln( \hat{\mathcal{L}}) + k \cdot \ln(N)$$
$\hat{\mathcal{L}}$ is the maximised value of the likelihood function of the model, $N$ is the number of datapoints and $k$ is the number of parameters estimated by the model. For least squares approximations, the likelihood function can be written as 
$$ \hat{\mathcal{L}} = \frac{S}{N}$$ where $S$ is the sum of squared differences:
$$S = \sum{\varepsilon_i^2} = \sum (y_i - \hat{y}_i)$$

Both, `AIC` and `BIC` punish models with many fit parameters. The smaller the `AIC` or `BIC` value, the better the model. In general, `BIC` favors a little bit less complex models than `AIC`.

### Exercise 10: Compute and plot RMSE, AIC and BIC

Repeat the computations of exercise 8, but this time calculate as well the `AIC` and `BIC` additionally to the root mean squared error (E). Print all values for all different degrees of the polynomials to the screen. Additionally, plot the `RMSE`, `AIC` and `BIC` in three subplots one below the other. Which model performs best according to the new criteria?

In [ ]:
x, y = np.loadtxt('data/xydatafit.dat')

d = np.arange(1,6)
RMSE = []
AIC = []
BIC = []

for i in d:
    p = np.polyfit(x, y, i)
    yfit = np.polyval(p, x)
    S = np.sum((yfit - y) ** 2)
   
    n = len(y)
    k = len(p)
    
    E = rmse(yfit, y)
    aic = n * np.log(S/n) + 2 * k
    bic = n * np.log(S/n) + k * np.log(n)
    
    RMSE.append(E)
    AIC.append(aic)
    BIC.append(bic)
    
    print(f'degree={i} => AIC={aic:.1f}; BIC={bic:.1f}')

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)

ax1.plot(d, RMSE, label='RMSE', color='k')
ax1.set_ylabel('RMSE')


ax2.plot(d, AIC, label='AIC', color='r')
ax2.set_ylabel('AIC')

ax3.plot(d, BIC, label='BIC')
plt.xlabel('degree of polynomial')
plt.ylabel('BIC')

ax1.legend()
ax2.legend()
ax3.legend();

## Fitting an arbitrary function
Python functions to fit a straight line or polynomial are readily available. There are many other functions that you may want to use to fit to your data. The function `curve_fit` can be used to fit an arbitrary function that you define; `curve_fit` is part of the `scipy.optimize` package. The `curve_fit` function requires you to write a function that takes as its first argument the independent variable (in our case above that is $x$) followed by the parameter(s) that you want to fit and the function must return the value of the function at all the $x$ values for the supplied parameters. For example, to fit a straight line, you need to write a function

In [ ]:
def func(x, a, b):
    return a * x + b

The `curve_fit` function needs to be called with three arguments: the function that you want to fit, the values of the independent variable (in our case $x$), and the values of the depenedent variable (in our case $y$). The `curve_fit` funtion than returns an array with the optimal parameters (in a least squares sense) and a second array containing the covariance of the optimal parameters (more on that later). For example, for the case of Exercise 1:

In [ ]:
from scipy.optimize import curve_fit
x, y = np.loadtxt('data/xydatafit.dat')  # in case these were modified in one of the exercises
popt, pcov = curve_fit(func, x, y)
print('optimal parameters:', popt)

Note that these optimal parameters are identical to the values you computed in Exercise 1. 

### Exercise 11. Fit exponential function to the data with a linear fit
Before we use the `curve_fit` function, we will perform a small trick to fit the exponential decay function
$$y = A \cdot exp(-a\cdot x )$$ 
to the data with the Python functions that we have used before. By transforming the exponential function through taking the logarithm
$$ln(y) = -a \cdot x + ln(A)$$
we are able to fit the data by taking the logarithm of the measurement values `y` and fit the result with through linear regression. Then we transform the intercept back with the exponential function to retrieve the $exp(ln(A)) = A$ parameter of the exponential decay function. 
Take the logarithm of the measurement values from the dataset found in file `xydatafit.dat`, fit a straight line with `linregress` and print the parameters `A`  and `-a` and the root mean squared error to the screen. Plot additionally the linear fit in the logarithmic scale and the backtransformed fitted exponential function on the original data in two subplots next to each other. Did the root mean squared error improve?

In [ ]:
x, y = np.loadtxt('data/xydatafit.dat')

slope, intercept, r_value, p_value, s_std = linregress(x, np.log(y))

y_fit = np.exp(intercept) * np.exp(slope * x)

E = rmse(yfit, y)

print('A=', np.exp(intercept))
print('-a=', slope)
print('RMSE=', E)

fig = plt.figure(figsize=(10,5))
ax1 = fig.add_subplot(121)

ax1.plot(x, np.log(y), 'ko', label='observed')
ax1.plot(x, np.log(y_fit), 'r-',  label='fitted')
plt.xlabel('x')
plt.ylabel('ln(y)')


ax2 = fig.add_subplot(122)
ax2.plot(x, y, 'ko', label='observed')
ax2.plot(x, y_fit, 'r-', label='fitted')
plt.xlabel('x')
plt.ylabel('y')


ax1.legend();
ax2.legend();


### Exercise 12. Fit exponential function with `curve_fit`

Now fit the data with Python scipy's `curve_fit` function, print again the optimal function parameters and the root mean squared error, `AIC` and `BIC`. Does this model perform better tan the polynomial fits? 

Note, that the fit parameters are slightly different between the linear exponential fit and the actual exponential fit, since the least square optimisation fits on slightly different functions.
The linear exponential fit minimises $ \sum_{i} \left( ln(y_i) - (ln(A) - a x)\right) $ the `curve_fit` minimises $ \sum_{i} \left( y_i - A \cdot exp(-a x_i)\right) $. 

In [ ]:
from scipy.optimize import curve_fit

def exponential(x, A, a):
    return A * np.exp(-a * x)

popt, pcov = curve_fit(exponential, x, y)
yfit = exponential(x, *popt)

n = len(y)
k = len(popt)
S = np.sum((yfit - y) ** 2)
E = rmse(yfit, y)
aic = n * np.log(S/n) + 2 * k
bic = n * np.log(S/n) + k * np.log(n)

print('Fit parameters:')
print('A=', popt[0])
print('a=', popt[1])

print('\nGoodness of fit criteria:')
print('RMSE=', E)
print('AIC=', aic)
print('BIC=', bic)

plt.plot(x, y, 'ko')
plt.plot(x, yfit, 'r-');

### Exercise 13. Fit a (extended) power law function with `curve_fit`
Use the $x,y$ values of 30 data points from the file `xydatafit.dat`. Fit the power law function $f_1(x) = a \ x^{-b}$ and the extended power law function $f_2(x) = (A + a \ x)^{-b}$  through the data using the `curve_fit` function of `scipy.optimize`. Plot the data and the two fitted function. Report RMSE, AIC and BIC for both fits. Did the RMSE, AIC and BIC values improve compared to the polynomial fits? Which function would you choose as the best fitting function? Take a closer look on covariance matrix of the fitted parameters (second output parameter of `curve_fit`) and look at the values and their standard deviation (square root of the variances in the diagonal of the covariance matrix) of the fit parameters. What does this tell us about the additional parameter `A`? Is it necessary to obtain a better model?

In [ ]:
from scipy.optimize import curve_fit

def ext_power_law(x, a, b, A):
    return (A + a * x) ** -b

def power_law(x, a, b):
    return a * x ** -b

plt.figure()
plt.plot(x, y, 'bo', label='observed')


for func in [power_law, ext_power_law]:

    popt, pcov = curve_fit(func, x, y)

    n = len(y)
    k = len(popt)
    yfit = func(x, *popt)
    S = np.sum((yfit - y) ** 2)
    
    E = rmse(yfit, y)
    aic = n * np.log(S/n) + 2 * k
    bic = n * np.log(S/n) + k * np.log(n)
    
    
    
    print('\t\t|RMSE\tAIC\tBIC')
    print('------------------------------------------')
    print(f'{func.__name__}\t{E:.4f}\t{aic:.2f}\t{bic:.2f}')
    print(f'\nFitparam:\ta, b, [A]=', popt )
    print(f'Std:\t\ta, b, [A]=', np.diag(pcov) )
    print()
    
    plt.plot(x, yfit, label='fit ' + func.__name__)
plt.legend(loc='best');

print('The additional parameter A is probably zero or close to zero with high uncertainties and hence not necessary for the fit')